In [1]:
import pandas as pd
import numpy as np # linear algebra

# Replace 'path_to_dataset' with the actual path to your dataset file
df = pd.read_excel('Dataset2neww.xlsx')
df.shape

(4424, 29)

In [2]:
X = df[['Application mode', 'Application order', 'Daytime/evening attendance\t',
       'Previous qualification', 'Admission grade', 'Displaced', 'Gender',
       'Age at enrollment', 'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (grade)',
       'Curricular units 2nd sem (without evaluations)', 'Unemployment rate',
       'Inflation rate', 'GDP', 'Target']]
y = df['Course']

categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['int', 'float']).columns

# Create DataFrames for categorical and numerical columns
X_categorical = X[categorical_columns]
X_numerical = X[numerical_columns]

from sklearn.preprocessing import LabelEncoder



# Perform label encoding only on columns with dtype object
label_encoder = LabelEncoder()
X_encoded = X_categorical.copy()
for col in categorical_columns:
    X_encoded[col] = X[col].astype(str)  # Ensure the column is of string dtype before label encoding
    X_encoded[col] = label_encoder.fit_transform(X_encoded[col])

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


# Initialize LabelEncoder for the target variable y
label_encoder_y = LabelEncoder()
y_encoded = label_encoder_y.fit_transform(y)
# Merge with original X containing int and float columns
X_final = pd.concat([X_encoded, X.select_dtypes(include=['int', 'float'])], axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_encoded, test_size=0.2, random_state=42)


In [3]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE

smote = SMOTE(random_state=42)
X_train_resampled_smote, y_train_resampled_smote = smote.fit_resample(X_train, y_train)

borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled_borderline, y_train_resampled_borderline = borderline_smote.fit_resample(X_train, y_train)


## BorutaShap SMOTE + Bordeline SMOTE
focus on Bordeline SMOTE only also can

RF

In [4]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the classifier
rf_classifier.fit(X_train_resampled_smote, y_train_resampled_smote)
# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_rf_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_rf = best_rf_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
classification_report_result_best_rf = classification_report(y_test, y_pred_best_rf, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Random Forest with SMOTE): {accuracy_best_rf}")
print("Classification Report (Best Random Forest with SMOTE):\n", classification_report_result_best_rf)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'max_depth': None, 'n_estimators': 200}
Accuracy (Best Random Forest with SMOTE): 0.5084745762711864
Classification Report (Best Random Forest with SMOTE):
               precision    recall  f1-score   support

           0      0.125     0.200     0.154         5
           1      0.630     1.000     0.773        34
           2      0.735     0.781     0.758        32
           3      0.487     0.780     0.600        50
           4      0.472     0.436     0.453        39
           5      0.433     0.382     0.406        68
           6      0.343     0.697     0.460        33
           7      0.438     0.179     0.255        39
           8      0.385     0.312     0.345        64
           9      0.487     0.507     0.497        75
          10      0.233     0.189     0.208        53
          11      0.720     0.712     0.716       163
          12      0.429     0.391     0.409        23
      

In [5]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize Borderline SMOTE

# Define the parameter grid for GridSearchCV
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_rf_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_rf = best_rf_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
classification_report_result_best_rf = classification_report(y_test, y_pred_best_rf, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Random Forest with Borderline SMOTE): {accuracy_best_rf}")
print("Classification Report (Best Random Forest with Borderline SMOTE):\n", classification_report_result_best_rf)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'max_depth': None, 'n_estimators': 200}
Accuracy (Best Random Forest with Borderline SMOTE): 0.5084745762711864
Classification Report (Best Random Forest with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.567     1.000     0.723        34
           2      0.758     0.781     0.769        32
           3      0.430     0.740     0.544        50
           4      0.462     0.462     0.462        39
           5      0.508     0.456     0.481        68
           6      0.333     0.545     0.414        33
           7      0.190     0.103     0.133        39
           8      0.424     0.438     0.431        64
           9      0.447     0.453     0.450        75
          10      0.216     0.151     0.178        53
          11      0.686     0.736     0.710       163
          12      0.400     0.174     

d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Gaussian NB

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-9, 1.778e-7, 3.162e-5, 5.6e-3, 1]
}

# Initialize the GaussianNB classifier
gnb_classifier = GaussianNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gnb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_gnb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_gnb = best_gnb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_gnb = accuracy_score(y_test, y_pred_best_gnb)
classification_report_result_best_gnb = classification_report(y_test, y_pred_best_gnb, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best GaussianNB with SMOTE): {accuracy_best_gnb}")
print("Classification Report (Best GaussianNB with SMOTE):\n", classification_report_result_best_gnb)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'var_smoothing': 3.162e-05}
Accuracy (Best GaussianNB with SMOTE): 0.25649717514124293
Classification Report (Best GaussianNB with SMOTE):
               precision    recall  f1-score   support

           0      0.027     0.200     0.048         5
           1      0.314     0.647     0.423        34
           2      0.692     0.562     0.621        32
           3      0.303     0.740     0.430        50
           4      1.000     0.026     0.050        39
           5      0.222     0.029     0.052        68
           6      0.250     0.273     0.261        33
           7      0.364     0.205     0.262        39
           8      0.333     0.016     0.030        64
           9      0.188     0.040     0.066        75
          10      0.333     0.019     0.036        53
          11      0.385     0.092     0.149       163
          12      0.062     0.043     0.051        23
          13      0.182  

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-9, 1.778e-7, 3.162e-5, 5.6e-3, 1]
}

# Initialize the GaussianNB classifier
gnb_classifier = GaussianNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gnb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_gnb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_gnb = best_gnb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_gnb = accuracy_score(y_test, y_pred_best_gnb)
classification_report_result_best_gnb = classification_report(y_test, y_pred_best_gnb, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best GaussianNB with SMOTE): {accuracy_best_gnb}")
print("Classification Report (Best GaussianNB with SMOTE):\n", classification_report_result_best_gnb)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'var_smoothing': 3.162e-05}
Accuracy (Best GaussianNB with SMOTE): 0.25875706214689265
Classification Report (Best GaussianNB with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.329     0.706     0.449        34
           2      0.667     0.562     0.610        32
           3      0.294     0.740     0.420        50
           4      1.000     0.026     0.050        39
           5      0.500     0.074     0.128        68
           6      0.238     0.303     0.267        33
           7      0.276     0.205     0.235        39
           8      0.000     0.000     0.000        64
           9      0.167     0.040     0.065        75
          10      0.250     0.038     0.066        53
          11      0.364     0.074     0.122       163
          12      0.056     0.087     0.068        23
          13      0.083  

DT

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None]
}

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_dt_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_dt = best_dt_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)
classification_report_result_best_dt = classification_report(y_test, y_pred_best_dt, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Decision Tree with SMOTE): {accuracy_best_dt}")
print("Classification Report (Best Decision Tree with SMOTE):\n", classification_report_result_best_dt)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'max_depth': None}
Accuracy (Best Decision Tree with SMOTE): 0.4090395480225989
Classification Report (Best Decision Tree with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.609     0.824     0.700        34
           2      0.649     0.750     0.696        32
           3      0.538     0.560     0.549        50
           4      0.308     0.410     0.352        39
           5      0.308     0.294     0.301        68
           6      0.360     0.545     0.434        33
           7      0.161     0.128     0.143        39
           8      0.254     0.266     0.260        64
           9      0.366     0.347     0.356        75
          10      0.178     0.245     0.206        53
          11      0.746     0.558     0.639       163
          12      0.333     0.304     0.318        23
          13      0.163     0

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None]
}

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_dt_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_dt = best_dt_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)
classification_report_result_best_dt = classification_report(y_test, y_pred_best_dt, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Decision Tree with SMOTE): {accuracy_best_dt}")
print("Classification Report (Best Decision Tree with SMOTE):\n", classification_report_result_best_dt)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'max_depth': None}
Accuracy (Best Decision Tree with SMOTE): 0.3751412429378531
Classification Report (Best Decision Tree with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.634     0.765     0.693        34
           2      0.625     0.625     0.625        32
           3      0.407     0.440     0.423        50
           4      0.158     0.154     0.156        39
           5      0.301     0.324     0.312        68
           6      0.349     0.455     0.395        33
           7      0.138     0.103     0.118        39
           8      0.279     0.297     0.288        64
           9      0.299     0.267     0.282        75
          10      0.129     0.170     0.146        53
          11      0.672     0.564     0.613       163
          12      0.227     0.217     0.222        23
          13      0.167     0

KNN


In [10]:
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume X_final and y_encoded are your features and target after preprocessing


# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE to the training data only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5]
}

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=knn_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_knn_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_knn = best_knn_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_knn = accuracy_score(y_test, y_pred_best_knn)
classification_report_result_best_knn = classification_report(y_test, y_pred_best_knn, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best KNN with SMOTE): {accuracy_best_knn}")
print("Classification Report (Best KNN with SMOTE):\n", classification_report_result_best_knn)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'n_neighbors': 1}
Accuracy (Best KNN with SMOTE): 0.3887005649717514
Classification Report (Best KNN with SMOTE):
               precision    recall  f1-score   support

           0      0.200     0.200     0.200         5
           1      0.714     0.882     0.789        34
           2      0.636     0.656     0.646        32
           3      0.489     0.440     0.463        50
           4      0.270     0.256     0.263        39
           5      0.406     0.382     0.394        68
           6      0.306     0.455     0.366        33
           7      0.250     0.179     0.209        39
           8      0.203     0.203     0.203        64
           9      0.309     0.333     0.321        75
          10      0.109     0.113     0.111        53
          11      0.551     0.564     0.558       163
          12      0.125     0.130     0.128        23
          13      0.156     0.175     0.165       

In [11]:
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume X_final and y_encoded are your features and target after preprocessing


# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Borderline SMOTE to the training data only
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5]
}

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=knn_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_knn_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_knn = best_knn_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_knn = accuracy_score(y_test, y_pred_best_knn)
classification_report_result_best_knn = classification_report(y_test, y_pred_best_knn, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best KNN with Borderline SMOTE): {accuracy_best_knn}")
print("Classification Report (Best KNN with Borderline SMOTE):\n", classification_report_result_best_knn)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'n_neighbors': 1}
Accuracy (Best KNN with Borderline SMOTE): 0.3887005649717514
Classification Report (Best KNN with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.698     0.882     0.779        34
           2      0.667     0.750     0.706        32
           3      0.489     0.440     0.463        50
           4      0.278     0.256     0.267        39
           5      0.371     0.382     0.377        68
           6      0.286     0.485     0.360        33
           7      0.269     0.179     0.215        39
           8      0.217     0.203     0.210        64
           9      0.293     0.293     0.293        75
          10      0.088     0.094     0.091        53
          11      0.532     0.558     0.545       163
          12      0.222     0.174     0.195        23
          13      0.152     

SVM

In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE



# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
}

# Initialize the SVM classifier
svm_classifier = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=svm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_svm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
classification_report_result_best_svm = classification_report(y_test, y_pred_best_svm, digits=3)

# Print results
print(f"Best Parameters (SVM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (SVM with SMOTE): {accuracy_best_svm}")
print("Classification Report (SVM with SMOTE):\n", classification_report_result_best_svm)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters (SVM with SMOTE): {'C': 100, 'gamma': 0.1}
Accuracy (SVM with SMOTE): 0.4271186440677966
Classification Report (SVM with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.689     0.912     0.785        34
           2      0.686     0.750     0.716        32
           3      0.448     0.520     0.481        50
           4      0.239     0.282     0.259        39
           5      0.352     0.456     0.397        68
           6      0.293     0.364     0.324        33
           7      0.120     0.077     0.094        39
           8      0.243     0.266     0.254        64
           9      0.372     0.427     0.398        75
          10      0.176     0.170     0.173        53
          11      0.630     0.638     0.634       163
          12      0.133     0.087     0.105        23
          13      0.212     0.175   

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = borderline_smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
}

# Initialize the SVM classifier
svm_classifier = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=svm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_svm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
classification_report_result_best_svm = classification_report(y_test, y_pred_best_svm, digits=3)

# Print results
print(f"Best Parameters (SVM with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (SVM with BorderlineSMOTE): {accuracy_best_svm}")
print("Classification Report (SVM with BorderlineSMOTE):\n", classification_report_result_best_svm)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters (SVM with BorderlineSMOTE): {'C': 100, 'gamma': 0.1}
Accuracy (SVM with BorderlineSMOTE): 0.43615819209039547
Classification Report (SVM with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.705     0.912     0.795        34
           2      0.676     0.719     0.697        32
           3      0.441     0.520     0.477        50
           4      0.260     0.333     0.292        39
           5      0.341     0.426     0.379        68
           6      0.279     0.364     0.316        33
           7      0.208     0.128     0.159        39
           8      0.221     0.234     0.227        64
           9      0.407     0.493     0.446        75
          10      0.239     0.208     0.222        53
          11      0.659     0.663     0.661       163
          12      0.267     0.174     0.211        23
     

LR

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],  # Choose either 'lbfgs' or 'liblinear' solver
}

# Initialize the Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if necessary

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lr_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_lr_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lr = best_lr_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_lr = accuracy_score(y_test, y_pred_best_lr)
classification_report_result_best_lr = classification_report(y_test, y_pred_best_lr, digits=3)

# Print results
print(f"Best Parameters (Logistic Regression with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (Logistic Regression with SMOTE): {accuracy_best_lr}")
print("Classification Report (Logistic Regression with SMOTE):\n", classification_report_result_best_lr)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters (Logistic Regression with SMOTE): {'C': 100, 'solver': 'lbfgs'}
Accuracy (Logistic Regression with SMOTE): 0.3254237288135593
Classification Report (Logistic Regression with SMOTE):
               precision    recall  f1-score   support

           0      0.062     0.200     0.095         5
           1      0.438     0.824     0.571        34
           2      0.676     0.781     0.725        32
           3      0.369     0.480     0.417        50
           4      0.194     0.179     0.187        39
           5      0.298     0.250     0.272        68
           6      0.272     0.667     0.386        33
           7      0.184     0.179     0.182        39
           8      0.182     0.031     0.053        64
           9      0.233     0.373     0.287        75
          10      0.300     0.113     0.164        53
          11      0.549     0.307     0.394       163
          12      0.127     0.391     

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],  # Choose either 'lbfgs' or 'liblinear' solver
}

# Initialize the Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if necessary

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lr_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_lr_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lr = best_lr_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_lr = accuracy_score(y_test, y_pred_best_lr)
classification_report_result_best_lr = classification_report(y_test, y_pred_best_lr, digits=3)

# Print results
print(f"Best Parameters (Logistic Regression with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (Logistic Regression with BorderlineSMOTE): {accuracy_best_lr}")
print("Classification Report (Logistic Regression with BorderlineSMOTE):\n", classification_report_result_best_lr)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters (Logistic Regression with BorderlineSMOTE): {'C': 100, 'solver': 'lbfgs'}
Accuracy (Logistic Regression with BorderlineSMOTE): 0.30847457627118646
Classification Report (Logistic Regression with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.438     0.824     0.571        34
           2      0.667     0.625     0.645        32
           3      0.324     0.480     0.387        50
           4      0.157     0.205     0.178        39
           5      0.304     0.206     0.246        68
           6      0.235     0.576     0.333        33
           7      0.208     0.256     0.230        39
           8      0.062     0.016     0.025        64
           9      0.248     0.387     0.302        75
          10      0.250     0.094     0.137        53
          11      0.506     0.270     0.352       163
       

XGBoost

In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, None],
}

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_xgb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_xgb = best_xgb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
classification_report_result_best_xgb = classification_report(y_test, y_pred_best_xgb, digits=3)

# Print results
print(f"Best Parameters (XGBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (XGBoost with SMOTE): {accuracy_best_xgb}")
print("Classification Report (XGBoost with SMOTE):\n", classification_report_result_best_xgb)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Parameters (XGBoost with SMOTE): {'gamma': None, 'max_depth': None}
Accuracy (XGBoost with SMOTE): 0.5412429378531074
Classification Report (XGBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.125     0.200     0.154         5
           1      0.681     0.941     0.790        34
           2      0.719     0.719     0.719        32
           3      0.617     0.580     0.598        50
           4      0.455     0.513     0.482        39
           5      0.507     0.500     0.504        68
           6      0.391     0.545     0.456        33
           7      0.316     0.154     0.207        39
           8      0.412     0.438     0.424        64
           9      0.533     0.533     0.533        75
          10      0.234     0.208     0.220        53
          11      0.730     0.828     0.776       163
          12      0.421     0.348     0.381        23
          13 

In [17]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, None],
}

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_xgb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_xgb = best_xgb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
classification_report_result_best_xgb = classification_report(y_test, y_pred_best_xgb, digits=3)

# Print results
print(f"Best Parameters (XGBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (XGBoost with SMOTE): {accuracy_best_xgb}")
print("Classification Report (XGBoost with SMOTE):\n", classification_report_result_best_xgb)


Fitting 5 folds for each of 30 candidates, totalling 150 fits


Best Parameters (XGBoost with SMOTE): {'gamma': None, 'max_depth': None}
Accuracy (XGBoost with SMOTE): 0.5435028248587571
Classification Report (XGBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.653     0.941     0.771        34
           2      0.710     0.688     0.698        32
           3      0.576     0.680     0.624        50
           4      0.415     0.436     0.425        39
           5      0.559     0.559     0.559        68
           6      0.525     0.636     0.575        33
           7      0.316     0.154     0.207        39
           8      0.451     0.500     0.474        64
           9      0.444     0.480     0.462        75
          10      0.295     0.245     0.268        53
          11      0.721     0.810     0.763       163
          12      0.312     0.217     0.256        23
          13      0.375     0.300     0.333        40
          14      0.4

In [18]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the XGBoost classifier with default parameters
xgb_classifier_baseline = XGBClassifier(random_state=42)

# Fit the classifier on the training data (assuming X_train and y_train are defined)
xgb_classifier_baseline.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb_baseline = xgb_classifier_baseline.predict(X_test)

# Evaluate the baseline model
accuracy_xgb_baseline = accuracy_score(y_test, y_pred_xgb_baseline)
classification_report_result_xgb_baseline = classification_report(y_test, y_pred_xgb_baseline, digits=3)

# Print results
print("Baseline XGBoost Model:")
print(f"Accuracy: {accuracy_xgb_baseline}")
print("Classification Report:\n", classification_report_result_xgb_baseline)


Baseline XGBoost Model:
Accuracy: 0.5559322033898305
Classification Report:
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.660     0.912     0.765        34
           2      0.793     0.719     0.754        32
           3      0.653     0.640     0.646        50
           4      0.500     0.513     0.506        39
           5      0.551     0.559     0.555        68
           6      0.450     0.545     0.493        33
           7      0.444     0.205     0.281        39
           8      0.459     0.531     0.493        64
           9      0.458     0.507     0.481        75
          10      0.244     0.208     0.224        53
          11      0.673     0.847     0.750       163
          12      0.667     0.174     0.276        23
          13      0.419     0.325     0.366        40
          14      0.475     0.372     0.417        78
          15      0.371     0.317     0.342        41
    

AdaBoost

In [19]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity


# Define the parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.1, 1],
}

# Initialize the AdaBoost classifier
ada_classifier = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=ada_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_ada_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_ada = best_ada_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_ada = accuracy_score(y_test, y_pred_best_ada)
classification_report_result_best_ada = classification_report(y_test, y_pred_best_ada, digits=3)

# Print results
print(f"Best Parameters (AdaBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (AdaBoost with SMOTE): {accuracy_best_ada}")
print("Classification Report (AdaBoost with SMOTE):\n", classification_report_result_best_ada)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters (AdaBoost with SMOTE): {'learning_rate': 0.1, 'n_estimators': 200}
Accuracy (AdaBoost with SMOTE): 0.28926553672316385
Classification Report (AdaBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.000     0.000     0.000        34
           2      0.568     0.781     0.658        32
           3      0.228     0.260     0.243        50
           4      0.133     0.103     0.116        39
           5      0.208     0.074     0.109        68
           6      0.148     0.636     0.240        33
           7      0.103     0.077     0.088        39
           8      0.000     0.000     0.000        64
           9      0.266     0.280     0.273        75
          10      0.000     0.000     0.000        53
          11      0.490     0.589     0.535       163
          12      0.096     0.304     0.146        23
 

In [20]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity


# Define the parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.1, 1],
}

# Initialize the AdaBoost classifier
ada_classifier = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=ada_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_ada_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_ada = best_ada_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_ada = accuracy_score(y_test, y_pred_best_ada)
classification_report_result_best_ada = classification_report(y_test, y_pred_best_ada, digits=3)

# Print results
print(f"Best Parameters (AdaBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (AdaBoost with SMOTE): {accuracy_best_ada}")
print("Classification Report (AdaBoost with SMOTE):\n", classification_report_result_best_ada)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters (AdaBoost with SMOTE): {'learning_rate': 0.1, 'n_estimators': 50}
Accuracy (AdaBoost with SMOTE): 0.2576271186440678
Classification Report (AdaBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.000     0.000     0.000        34
           2      0.491     0.844     0.621        32
           3      0.140     0.140     0.140        50
           4      0.071     0.026     0.038        39
           5      0.167     0.074     0.102        68
           6      0.151     0.818     0.255        33
           7      0.188     0.077     0.109        39
           8      0.000     0.000     0.000        64
           9      0.385     0.133     0.198        75
          10      0.069     0.038     0.049        53
          11      0.418     0.503     0.457       163
          12      0.075     0.217     0.111        23
   

In [21]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the AdaBoost classifier with default parameters
ada_classifier_baseline = AdaBoostClassifier(random_state=42)

# Fit the classifier on the training data (assuming X_train and y_train are defined)
ada_classifier_baseline.fit(X_train, y_train)

# Make predictions on the test set
y_pred_ada_baseline = ada_classifier_baseline.predict(X_test)

# Evaluate the baseline model
accuracy_ada_baseline = accuracy_score(y_test, y_pred_ada_baseline)
classification_report_result_ada_baseline = classification_report(y_test, y_pred_ada_baseline, digits=3)

# Print results
print("Baseline AdaBoost Model:")
print(f"Accuracy: {accuracy_ada_baseline}")
print("Classification Report:\n", classification_report_result_ada_baseline)


Baseline AdaBoost Model:
Accuracy: 0.29265536723163843
Classification Report:
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.420     0.853     0.563        34
           2      0.727     0.250     0.372        32
           3      0.000     0.000     0.000        50
           4      0.240     0.154     0.188        39
           5      0.111     0.044     0.063        68
           6      0.000     0.000     0.000        33
           7      0.000     0.000     0.000        39
           8      0.000     0.000     0.000        64
           9      0.274     0.227     0.248        75
          10      0.000     0.000     0.000        53
          11      0.246     0.926     0.389       163
          12      0.000     0.000     0.000        23
          13      0.000     0.000     0.000        40
          14      0.000     0.000     0.000        78
          15      0.000     0.000     0.000        41
  

CatBoost

LGBM

In [22]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for LGBM
param_grid = {
    'max_depth': [-1, 2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the LGBM classifier
lgbm_classifier = LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Assuming X_train_resampled_smote, y_train_resampled_smote are defined
# Fit GridSearchCV on SMOTE resampled data
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_lgbm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lgbm = best_lgbm_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_lgbm = accuracy_score(y_test, y_pred_best_lgbm)
classification_report_result_best_lgbm = classification_report(y_test, y_pred_best_lgbm, digits=3)

# Print results
print(f"Best Parameters (LGBM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (LGBM with SMOTE): {accuracy_best_lgbm}")
print("Classification Report (LGBM with SMOTE):\n", classification_report_result_best_lgbm)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001519 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1527
[LightGBM] [Info] Number of data points in the train set: 10251, number of used features: 18
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] Start training from score -2.833213
[LightGBM] [Info] S

In [23]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for LGBM
param_grid = {
    'max_depth': [-1, 2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the LGBM classifier
lgbm_classifier = LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Assuming X_train_resampled_smote, y_train_resampled_smote are defined
# Fit GridSearchCV on SMOTE resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_lgbm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lgbm = best_lgbm_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_lgbm = accuracy_score(y_test, y_pred_best_lgbm)
classification_report_result_best_lgbm = classification_report(y_test, y_pred_best_lgbm, digits=3)

# Print results
print(f"Best Parameters (LGBM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (LGBM with SMOTE): {accuracy_best_lgbm}")
print("Classification Report (LGBM with SMOTE):\n", classification_report_result_best_lgbm)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007021 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1500
[LightGBM] [Info] Number of data points in the train set: 9655, number of used features: 18
[LightGBM] [Info] Start training from score -7.229321
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] St